# ALGORITHM 2



In [4]:
import numpy as np
import timeit
n=100
sigma = 1000
def generate_data():
    l = np.random.uniform(10, 50, size=n)
    c = np.random.uniform(10, 50, size=n)
    x_bar = np.random.uniform(1, 10, size=n)
    #y_bar = np.random.uniform(1, c-1, size=n)
    y_bar = [np.random.uniform(1, c_i - 1) for c_i in c]
    lbd = np.random.uniform(5, 10, size=n)
    mu = np.random.uniform(1, 150, size=n)
    input_data= {
        "l": l,
        "c": c,
        "x_bar": x_bar,
        "y_bar": y_bar,
        "lbd": lbd,
        "mu": mu,
    }
    return input_data
input_data=generate_data()
Budget = np.sum(input_data["lbd"] + input_data["mu"])/100
def calculate_T(input_data):
    l=input_data["l"]
    c=input_data["c"]
    x_bar=input_data["x_bar"]
    y_bar=input_data["y_bar"]
    lbd = input_data["lbd"]
    mu=input_data["mu"]
    T = l + sigma/c
    T_max = l+ x_bar + sigma/(c - y_bar)
    T_x = l+ x_bar + sigma/c
    T_y = l + sigma/(c - y_bar)
    output_T= {
        "T": T,
        "T_max": T_max,
        "T_x": T_x,
        "T_y": T_y,

    }
    return output_T
output_T=calculate_T(input_data)

In [5]:
def algorithm_alg2(input_data,output_T,Budget):
  alpha=np.min(output_T["T"])
  beta=np.min(output_T["T_max"])
  def determine_indices_alg2(input_data,output_T):
      lbd=input_data["lbd"]
      mu=input_data["mu"]
      x_bar=input_data["x_bar"]
      T_x=output_T["T_x"]
      T_y=output_T["T_y"]
      I_1 = np.where(( lbd <= mu) & (T_y <= T_x ))[0]
      I_2 = np.where((lbd <= mu) & (T_y > T_x))[0]
      I_3 = np.where(( lbd > mu) & (T_x <= T_y ))[0]
      I_4 = np.where(( lbd > mu) & (T_x > T_y ))[0]
      output_indices_alg2={
          "I_1":I_1,
          "I_2":I_2,
          "I_3":I_3,
          "I_4":I_4,
      }
      return output_indices_alg2
  output_indices_alg2 = determine_indices_alg2(input_data,output_T)
  def C_breakpoint_alg2(i,output_T,output_indices_alg2):
      T_x=output_T["T_x"]
      T_y=output_T["T_y"]
      I_1=output_indices_alg2["I_1"]
      I_2=output_indices_alg2["I_2"]
      I_3=output_indices_alg2["I_3"]
      I_4=output_indices_alg2["I_4"]
      C_break=[]
      if i in I_1:
        C_break.append(T_x[i])
      elif i in I_2:
        C_break.append(T_x[i])
        C_break.append(T_y[i])
      elif i in I_3:
        C_break.append(T_y[i])
      elif i in I_4:
        C_break.append(T_x[i])
        C_break.append(T_y[i])
      return C_break
  all_C_breakpoints_alg2 = []
  for i in range(len(input_data["l"])):
      all_C_breakpoints_alg2.append(C_breakpoint_alg2(i, output_T, output_indices_alg2))
  K = set()
  for sublist in all_C_breakpoints_alg2:
      K.update(sublist)
  K.update(output_T["T"])
  K.update([alpha,beta])
  break_points_alg2= np.array(list(K))
  K = []
  for t in break_points_alg2 :
      if alpha<=t<= beta:
          K.append(t)
  K= np.sort(K)
# Cost function
  def get_cost(i, t, input_data, output_T, output_indices_alg2):
      lbd = input_data["lbd"]
      mu = input_data["mu"]
      T=output_T["T"]
      T_x=output_T["T_x"]
      T_y=output_T["T_y"]
      I_1=output_indices_alg2["I_1"]
      I_2=output_indices_alg2["I_2"]
      I_3=output_indices_alg2["I_3"]
      I_4=output_indices_alg2["I_4"]
      # Xử lý tính toán giá trị cost
      if i in I_1:
          if t<= T[i]:
              cost = 0
          elif T[i] < t <= T_x[i]:
              cost = lbd[i]
          else:
              cost = mu[i] + lbd[i]
      elif i in I_2:
          if t<= T[i]:
              cost = 0
          elif T[i] < t <= T_x[i]:
              cost = lbd[i]
          elif T_x[i] < t <= T_y[i]:
              cost=mu[i]
          else:
              cost = mu[i] + lbd[i]
      elif i in I_3:
          if t<= T[i]:
              cost = 0
          elif T[i] < t <= T_y[i]:
              cost = mu[i]
          else:
              cost = mu[i] + lbd[i]
      elif i in I_4:
          if t<= T[i]:
              cost = 0
          elif T[i] <t <= T_y[i]:
              cost = mu[i]
          elif T_y[i] < t <= T_x[i]:
              cost=lbd[i]
          else:
              cost = mu[i] + lbd[i]
      return cost
# Total cost
  def total_cost_alg2(t, input_data, output_T, output_indices_alg2):
      total_cost = 0
      for i in range(len(input_data["l"])):
          cost = get_cost(i, t, input_data, output_T, output_indices_alg2)
          total_cost += cost
      return total_cost
  def find_optimal_solution_alg2(Budget, K, input_data, output_T, output_indices_alg2):
      r = 1
      s = len(K)
      optimal_solution = None

      while r < s:
          i = (r + s) // 2
          Omega_i = total_cost_alg2(t=K[i], input_data=input_data, output_T=output_T, output_indices_alg2=output_indices_alg2)

          if Omega_i < Budget:
              r = i + 1
              optimal_solution = i
          else:
              s = i

      # Kiểm tra vị trí tối ưu đã lưu
      if optimal_solution is not None:
          t_optimal = K[optimal_solution]
          Omega_optimal = total_cost_alg2(t=t_optimal, input_data=input_data, output_T=output_T, output_indices_alg2=output_indices_alg2)
          return t_optimal
      else:
          return None
  t_optimal_alg2= find_optimal_solution_alg2(Budget,K,input_data=input_data,output_T=output_T,output_indices_alg2=output_indices_alg2)
#Return set of indices
  def update_indices_alg2(t,group, output_T, output_indices_alg2):
      """
      group = "I_1", "I_2", "I_3", "I_4"
      """

      T = output_T["T"]
      I_1 = output_indices_alg2["I_1"]
      I_2 = output_indices_alg2["I_2"]
      I_3 = output_indices_alg2["I_3"]
      I_4 = output_indices_alg2["I_4"]
      if group == "I_1":
          # i in IU: U <= t
          res = [i for i in I_1 if T[i] < t]
      elif group == "I_2":
          # i in IU: U <= t
          res = [i for i in I_2 if T[i] < t]
      elif group == "I_3":
          # i in IU: U <= t
          res = [i for i in I_3 if T[i] < t]
      elif group == "I_4":
          # i in IU: U <= t
          res = [i for i in I_4 if T[i] < t]
      return res
  I_1_update_alg2=update_indices_alg2(t=t_optimal_alg2, group="I_1",output_T=output_T, output_indices_alg2=output_indices_alg2)
  I_2_update_alg2=update_indices_alg2(t=t_optimal_alg2, group="I_2",output_T=output_T, output_indices_alg2=output_indices_alg2)
  I_3_update_alg2=update_indices_alg2(t=t_optimal_alg2, group="I_3",output_T=output_T, output_indices_alg2=output_indices_alg2)
  I_4_update_alg2=update_indices_alg2(t=t_optimal_alg2, group="I_4",output_T=output_T, output_indices_alg2=output_indices_alg2)
#Return parameters
  def solution_for_group(i, t, output_T, input_data, group):
      T = output_T["T"]
      T_x = output_T["T_x"]
      T_y = output_T["T_y"]
      x_bar = input_data["x_bar"]
      y_bar = input_data["y_bar"]

      if group == "I_1_update_alg2":
          if t <= T_x[i]:
              x_i = x_bar[i]
              y_i = 0
          elif t > T_x[i]:
              x_i = x_bar[i]
              y_i = y_bar[i]
      elif group == "I_2_update_alg2":
          if t <= T_x[i]:
              x_i = x_bar[i]
              y_i = 0
          elif T_x[i] < t <= T_y[i]:
              x_i = 0
              y_i= y_bar[i]
          elif t > T_y[i]:
              x_i = x_bar[i]
              y_i= y_bar[i]
      elif group == "I_3_update_alg2":
          if t <= T_y[i]:
              x_i = 0
              y_i = y_bar[i]
          elif t > T_y[i]:
              x_i = x_bar[i]
              y_i = y_bar[i]
      elif group == "I_4_update_alg2":
          if t <= T_y[i]:
              x_i = 0
              y_i = y_bar[i]
          elif T_y[i] < t <= T_x[i]:
              x_i = x_bar[i]
              y_i = 0
          elif t > T_x[i]:
              x_i = x_bar[i]
              y_i = y_bar[i]
      return x_i, y_i
  x = np.zeros_like(input_data["l"])
  y = np.zeros_like(input_data["l"])
  for i in I_1_update_alg2:
      x_i, y_i = solution_for_group(i, t=t_optimal_alg2, output_T=output_T, input_data=input_data, group="I_1_update_alg2")
      x[i] = x_i
      y[i] = y_i
  for i in I_2_update_alg2:
      x_i, y_i = solution_for_group(i, t=t_optimal_alg2, output_T=output_T, input_data=input_data, group="I_2_update_alg2")
      x[i] = x_i
      y[i] = y_i
  for i in I_3_update_alg2:
      x_i, y_i = solution_for_group(i, t=t_optimal_alg2, output_T=output_T, input_data=input_data, group="I_3_update_alg2")
      x[i] = x_i
      y[i] = y_i
  for i in I_4_update_alg2:
      x_i, y_i = solution_for_group(i, t=t_optimal_alg2, output_T=output_T, input_data=input_data, group="I_4_update_alg2")
      x[i] = x_i
      y[i] = y_i
  return  x,y
result_alg2= algorithm_alg2(input_data,output_T,Budget)

In [6]:
# Number of runs
num_runs = 50

# Calculate the running time of ALgorithm 2
times_alg2 = timeit.repeat(lambda: algorithm_alg2(input_data,output_T,Budget), number=1, repeat=num_runs)

# Average running time of Algorithm 2
time_avg_alg2 = sum(times_alg2) / num_runs
print(time_avg_alg2)